# Blockhouse Work Trial Task-2

**Modeling Temporary Impact Function and Optimizing Order Execution**

## Problem Overview

We aim to understand the temporary impact of market orders using order book data from 3 tickers (CRWV, FROG, SOUN). The main goals are:

1. **Model the temporary impact function** \( g_t(x) \) — slippage due to buying \( x \) shares at time \( t \).
2. **Design execution strategies** to buy \( S = 10,000 \) shares over a trading day (\( N = 390 \) periods), minimizing total slippage.

---


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

plt.style.use('seaborn-v0_8-notebook')

In [ ]:
def compute_slippage(ask_prices, ask_sizes, x, bid_price, ask_price):
    cumulative_size = 0
    cost = 0
    for px, sz in zip(ask_prices, ask_sizes):
        if cumulative_size + sz <= x:
            cost += px * sz
            cumulative_size += sz
        else:
            remaining = x - cumulative_size
            cost += px * remaining
            cumulative_size += remaining
            break
    if cumulative_size < x:
        return np.nan  # not enough liquidity
    avg_price = cost / x
    mid_price = (bid_price + ask_price) / 2
    return avg_price - mid_price

In [ ]:
# Sample from CRWV
ask_px = [55.4, 55.46, 55.47, 55.5, 55.93, 56.0, 56.24, 56.45, 56.66, 56.7]
ask_sz = [199, 10, 100, 100, 50, 200, 3, 5, 4, 10]
bid_px = [55.1, 55.06, 55.03, 55.00, 54.98, 54.80, 54.78, 54.66, 54.57, 54.55]

buy_sizes = np.arange(10, 700, 10)
slippage_values = [compute_slippage(ask_px, ask_sz, x, bid_px[0], ask_px[0]) for x in buy_sizes]

plt.figure(figsize=(10,5))
plt.plot(buy_sizes, slippage_values, label="Temporary Impact g(x)", color="orange")
plt.xlabel("Buy Size x")
plt.ylabel("Slippage ($)")
plt.title("Temporary Impact Function g(x)")
plt.grid(True)
plt.legend()
plt.show()

## Strategy 1: Equal Allocation
Buy $S/390$ shares uniformly across all periods.

In [ ]:
S = 10000
N = 390
allocation_equal = np.full(N, S // N)

plt.plot(allocation_equal)
plt.title("Equal Allocation Strategy")
plt.xlabel("Minute")
plt.ylabel("Shares Bought")
plt.grid(True)
plt.show()

## Strategy 2: Greedy Allocation
Buy more shares in minutes with lower slippage (using simulated slippage values).

In [ ]:
# Simulated slippage (lower is better)
np.random.seed(42)
simulated_slippage = np.sort(np.random.rand(N))
inverse_slip = 1 / (simulated_slippage + 0.01)
allocation_greedy = (inverse_slip / inverse_slip.sum()) * S

plt.plot(allocation_greedy)
plt.title("Greedy Allocation Based on Simulated Slippage")
plt.xlabel("Minute")
plt.ylabel("Shares Bought")
plt.grid(True)
plt.show()